In [9]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string

In [16]:
from itertools import combinations

# Subject group data and student counts
subject_groups = {
    ('ML', 'IS', 'CC', 'DF'): 69,
    ('BDA', 'IS', 'MCWC', 'MAD'): 11,
    ('BDA', 'IS', 'CC', 'DF'): 78,
    ('BDA', 'IS', 'CC', 'MAD'): 7,
    ('BDA', 'IS', 'MCWC', 'DF'): 13,
}

# Initial subject hours
total_hours = {
    "ML": 5,
    "BDA": 5,
    "IS": 5,
    "MCWC": 3,
    "CC": 3,
    "MAD": 4,
    "DF": 4
}

# Constraints
min_students = 0  # No minimum size for divisions
max_students = 60  # Maximum size per division


def allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students):
    groups = list(subject_groups.keys())
    students = list(subject_groups.values())
    best_solution = {"divisions": None, "deviation": float("inf")}
    recursive_calls = 0

    def calculate_hours(division):
        """Calculate total hours per subject in a division."""
        hours = {subject: 0 for subject in total_hours}
        for group in division:
            for subject in group:
                hours[subject] = total_hours[subject]
        return hours

    def calculate_deviation(hours):
        """Calculate the total deviation of hours from the initial requirement."""
        return sum(abs(hours[sub] - total_hours[sub]) for sub in total_hours)

    def backtrack(current_divisions, current_students, remaining_groups, remaining_counts):
        nonlocal recursive_calls
        recursive_calls += 1

        # Base case: no groups left to allocate
        if not remaining_groups:
            total_hours_all = {subject: 0 for subject in total_hours}
            for division in current_divisions:
                hours = calculate_hours(division)
                for subject in total_hours_all:
                    total_hours_all[subject] += hours[subject]
            deviation = calculate_deviation(total_hours_all)
            if deviation < best_solution["deviation"]:
                best_solution["divisions"] = [div[:] for div in current_divisions]
                best_solution["deviation"] = deviation
            return

        # Get the next group and its student count
        group = remaining_groups[0]
        count = remaining_counts[0]

        # Split the group if necessary
        while count > max_students:
            current_divisions.append([group])
            current_students.append(max_students)
            count -= max_students

        # Try allocating the remaining part of the group to each division
        for i in range(len(current_divisions)):
            if current_students[i] + count <= max_students:
                current_divisions[i].append(group)
                current_students[i] += count
                backtrack(
                    current_divisions,
                    current_students,
                    remaining_groups[1:],
                    remaining_counts[1:],
                )
                # Backtrack
                current_divisions[i].pop()
                current_students[i] -= count

        # Start a new division for the remaining part
        current_divisions.append([group])
        current_students.append(count)
        backtrack(
            current_divisions,
            current_students,
            remaining_groups[1:],
            remaining_counts[1:],
        )
        # Backtrack
        current_divisions.pop()
        current_students.pop()

    # Initialize backtracking
    backtrack([], [], groups, students)

    # Log statistics
    print(f"Total Recursive Calls: {recursive_calls}")

    # Calculate the final subject hours for each subject
    total_hours_all_divisions = {subject: 0 for subject in total_hours}
    for division in best_solution["divisions"]:
        division_hours = calculate_hours(division)
        for subject in total_hours_all_divisions:
            total_hours_all_divisions[subject] += division_hours[subject]

    # Return the best solution with subject hours included
    return best_solution, total_hours_all_divisions


# Run the revised algorithm
result, subject_hours = allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students)

# Display results
print("\nOptimal Division Allocation:")
if result["divisions"]:
    for i, division in enumerate(result["divisions"], start=1):
        print(f"Division {i}: {division}")

# Print total deviation
print("\nTotal Deviation:", result["deviation"])

# Print the subject hours for each subject in the final solution
print("\nFinal Subject Hours Across All Divisions:")
for subject, hours in subject_hours.items():
    print(f"{subject}: {hours} hours")

<QuerySet [<Subject: Information security | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Machine Learning | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Application Development | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Computing and Wireless Communication | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>]>

In [12]:
# Sample data for student groups and their counts
student_groups =  {
    ('ML', 'IS', 'CC', 'DF') : 69,
    ('ML', 'IS', 'CC', 'MAD') : 44,
    ('ML', 'IS', 'MCWC', 'MAD') : 5,
    ('BDA', 'IS', 'MCWC', 'MAD') : 19,
    ('BDA', 'IS', 'CC', 'MAD') : 25,
    ('BDA', 'IS', 'CC', 'DF') : 16
}


# Initial subject hours
total_hours = {
    "ML": 5,
    "BDA": 5,
    "IS": 5,
    "MCWC": 3,
    "CC": 3,
    "MAD": 4,
    "DF": 4
}

# Max students per division
MAX_STUDENTS_PER_DIVISION = 90

# Function to split groups if the count exceeds the max allowed per division
def split_groups(groups, max_students):
    split_groups = {}
    for group, count in groups.items():
        part = 1
        while count > max_students:
            split_groups[(group, f"part{part}")] = max_students
            count -= max_students
            part += 1
        if count > 0:
            split_groups[(group, f"part{part}")] = count
    return split_groups

# Split the student groups if any group exceeds MAX_STUDENTS_PER_DIVISION
student_groups = split_groups(student_groups, MAX_STUDENTS_PER_DIVISION)

# Calculate the number of divisions needed based on total students
total_students = sum(student_groups.values())
num_divisions = (total_students // MAX_STUDENTS_PER_DIVISION) + (1 if total_students % MAX_STUDENTS_PER_DIVISION != 0 else 0)

# Initialize division allocations
divisions = {i: [] for i in range(num_divisions)}  # A list of student groups for each division
division_sizes = {i: 0 for i in range(num_divisions)}  # Number of students in each division

# Initialize teaching hours for each subject in each division
teaching_hours_per_division = {i: {subject: 0 for subject in total_hours} for i in range(num_divisions)}

# Backtracking function to try placing groups in divisions
def backtrack(group_idx, student_groups, division_sizes, divisions, teaching_hours_per_division):
    # Base case: All groups are assigned
    if group_idx == len(student_groups):
        # Check if all subjects meet the required hours in each division
        for subject, min_hours in total_hours.items():
            total_hours_assigned = sum(teaching_hours_per_division[d][subject] for d in range(num_divisions))
            if total_hours_assigned < min_hours:
                return False
        return True

    group, student_count = list(student_groups.items())[group_idx]
    subjects = group[0]  # group[0] is the original tuple of subjects
    
    # Try placing this group in any division (while respecting the max student limit)
    for div in range(num_divisions):
        if division_sizes[div] + student_count <= MAX_STUDENTS_PER_DIVISION:
            # Tentatively add the group to the division
            division_sizes[div] += student_count
            for subject in subjects:
                teaching_hours_per_division[div][subject] += student_count
            
            # Recurse to place the next group
            if backtrack(group_idx + 1, student_groups, division_sizes, divisions, teaching_hours_per_division):
                # If valid, assign this group to the division
                divisions[div].append((group, student_count))
                return True
            
            # If placing the group here didn't work, backtrack
            division_sizes[div] -= student_count
            for subject in subjects:
                teaching_hours_per_division[div][subject] -= student_count

    return False

# Call the backtracking function to find a solution
subjects_in_divs = {}
if backtrack(0, student_groups, division_sizes, divisions, teaching_hours_per_division):
    print("Optimal Assignment of Student Groups to Divisions:")
    for div in range(num_divisions):
        print(f"Division {div + 1}:")
        if f"div_{div + 1}" not in subjects_in_divs:
            subjects_in_divs[f"div_{div + 1}"] = {}
        for group, student_count in divisions[div]:
            for sub in group[0]:
                subjects_in_divs[f"div_{div + 1}"][sub]=1
            print(f"  Group {group[0]} ({group[1]}) with {student_count} students")
                    
    # print("\nFinal Teaching Hours per Subject:")
    # for subject in total_hours:
    #     total_hours_assigned = sum(teaching_hours_per_division[d][subject] for d in range(num_divisions))
    #     print(f"{subject}: {total_hours_assigned} hours (Minimum required: {total_hours[subject]} hours)")
else:
    print("No valid assignment found.")

# Initialize a final dictionary
merged_subject_count = {}

# Iterate over each division's dictionary
for div in subjects_in_divs.values():
    for subject, count in div.items():
        # Add the count to the corresponding subject in the merged dictionary
        merged_subject_count[subject] = merged_subject_count.get(subject, 0) + count
print()
print("Final Subject Hours Across All Divisions:")
for sub in merged_subject_count:
    print(f"{sub} : {merged_subject_count[sub] * total_hours[sub]} hours")

Optimal Assignment of Student Groups to Divisions:
Division 1:
  Group ('BDA', 'IS', 'MCWC', 'DF') (part1) with 13 students
  Group ('BDA', 'IS', 'CC', 'MAD') (part1) with 7 students
  Group ('ML', 'IS', 'CC', 'DF') (part1) with 69 students
Division 2:
  Group ('BDA', 'IS', 'CC', 'DF') (part1) with 78 students
  Group ('BDA', 'IS', 'MCWC', 'MAD') (part1) with 11 students

Final Subject Hours Across All Divisions:
BDA : 10 hours
IS : 10 hours
MCWC : 6 hours
DF : 8 hours
CC : 6 hours
MAD : 8 hours
ML : 5 hours
